# <span style="color:blue">Importing libraries</span>

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import requests
import os.path
import re

# <span style="color:blue">Data reading from web</span>

### <span style="color:red">The code below takes quite long to be run and sometimes it hangs.</span>  

It is for illustration of initial data collection only. **For practice, the saved csv files should be used.**

### Reading tables arranged by UFO observation date

This returns a list of dataframes, the `[0]` element being a dataframe describing the tables of UFO observations by date. The `Reports` column contains the list of the available tables.  
The dataframes corresponding to each shape can then be retrieved from
`http://www.nuforc.org/webreports/ndxs{date}.html`. The `{date}` in the url should be empty for `Unspecified` date, otherwise in the `YYYYMM` format.

In [ ]:
tables_by_date_list = pd.read_html('http://www.nuforc.org/webreports/ndxevent.html')

The code below saves each table as local html file for further import if necessary. This is done because reading of numerous files from the server is unstable and sometimes fails. Saving the html files can be restarted if needed - only the missing files will be downloaded during subsequent runs. Overall, **the downloading takes about 25 minutes**.

In [ ]:
if not os.path.exists('by_date'):
        os.mkdir('by_date')

for date in tables_by_date_list[0].Reports:
    if date == 'UNSPECIFIED / APPROXIMATE':
        url = 'http://www.nuforc.org/webreports/ndxe.html'
        file = 'by_date/unspecified.html'
    else:
        url = f'http://www.nuforc.org/webreports/ndxe{date[-4:]}{date[:2]}.html'
        file = 'by_date/' + date.replace('/', '_') + '.html'
    
    if not os.path.isfile(file):
        r = requests.get(url, allow_redirects=True)
        open(file, 'wb').write(r.content)
        # optional text output to track the process
        #print(file)

The code below is for reading the tables from url directly to the dataframe. If often fails; when successful, **<ins>the reading takes about an hour</ins>**

In [ ]:
ufo_by_date = []

for date in tables_by_date_list[0].Reports:
    #print(f'Reading data for shape = {shape_name}...')
    if date == 'UNSPECIFIED / APPROXIMATE':
        temp_df = pd.read_html(f'http://www.nuforc.org/webreports/ndxe.html')[0]
        temp_df['Date_label'] = date
        ufo_by_date.append(temp_df)
    else:
        temp_df = pd.read_html(f'http://www.nuforc.org/webreports/ndxe{date[-4:]}{date[:2]}.html')[0]
        temp_df['Date_label'] = date
        ufo_by_date.append(temp_df)
    # optional text output to monitor the process
    #print(f'Reading data for date = {date} done!')

finish_reading_by_date = pd.Timestamp.now()    
    
ufo_raw_date = pd.concat(ufo_by_date)

Saving the dataframe for further use

In [ ]:
ufo_raw_date.reset_index(drop=True, inplace=True)

In [ ]:
ufo_raw_date.to_csv("ufo_raw_date.csv")

### Reading tables arranged by UFO shape

This code returns a list of dataframes, the `[0]` element being a dataframe showing UFOs by type.  
The dataframes corresponding to each shape can then be retrieved from
`'http://www.nuforc.org/webreports/ndxs{shape_name}.html'`. The `{shape}` in the url should be empty for `Unspecified` shape.

In [ ]:
tables_by_shape_list = pd.read_html('http://www.nuforc.org/webreports/ndxshape.html')

Subsequent code is similar to reading by date. The data reading to dataframe takes **<ins>about 20 minutes</ins>** and is not always stable. The data reading to file takes about **4 minutes**.

In [ ]:
if not os.path.exists('by_shape'):
        os.mkdir('by_shape')

for shape_name in tables_by_shape_list[0].Reports:
    if shape_name == 'Unspecified':
        url = 'http://www.nuforc.org/webreports/ndxs.html'
        file = 'by_shape/unspecified.html'
    else:
        url = f'http://www.nuforc.org/webreports/ndxs{shape_name}.html'
        file = 'by_shape/' + shape_name + '.html'
    
    if not os.path.isfile(file):
        r = requests.get(url, allow_redirects=True)
        open(file, 'wb').write(r.content)
        # optional text output to monitor the process
        print(file)

In [ ]:
ufo_by_shape = []

for shape_name in tables_by_shape_list[0].Reports:
    if shape_name == 'Unspecified':
        temp_df = pd.read_html(f'http://www.nuforc.org/webreports/ndxs.html')[0]
    else:
        temp_df = pd.read_html(f'http://www.nuforc.org/webreports/ndxs{shape_name}.html')[0]
    ufo_by_shape.append(temp_df)
    # optional text output to monitor the process
    #print(f'{shape_name} done!')

ufo_raw_shape = pd.concat(ufo_by_shape)

In [ ]:
ufo_raw_shape.to_csv("ufo_raw_shape.csv")

### Reading information about the states (USA and Canada)

Taking information from wikipedia: [USA](https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States) and [Canada](https://en.wikipedia.org/wiki/Provinces_and_territories_of_Canada')

We will need the states population and area (total and land ones) for further analysis.

In [2]:
USA_wiki = 'https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States'

tables_states_list = pd.read_html(USA_wiki)

The first table about the USA states

In [22]:
states = tables_states_list[0]
states.head(1)

Flag, name andpostal abbreviation[12]  \
  Flag, name andpostal abbreviation[12]   
0                               Alabama   

                                               Cities              \
  Flag, name andpostal abbreviation[12].1     Capital Largest[16]   
0                                      AL  Montgomery  Birmingham   

  Ratification oradmission[C] Population[14] Total area[15]          \
  Ratification oradmission[C] Population[14]            mi2     km2   
0                Dec 14, 1819        5024279          52420  135767   

  Land area[15]         Water area[15]       Numberof Reps.  
            mi2     km2            mi2   km2 Numberof Reps.  
0         50645  131171           1775  4597              7

In [23]:
%%capture

# Leaving area in km^2
states = states.iloc[:, [0, 1, 5, 7, 9]]
states.columns = ['Name', 'Abbreviation', 'Population', 'Total_area', 'Land_area']

# Removing a note mark in the names
states.Name = states.Name.str.replace('[D]', '', regex=False)

states.Population = states.Population.astype('int64')

For further merging, the `Population`, `Total_area`, and `Land_area` should be numeric.

In [24]:
states.dtypes

Name            object
Abbreviation    object
Population       int64
Total_area       int64
Land_area        int64
dtype: object

Adding information of District of Columbia

In [25]:
DC = tables_states_list[1]

DC

Name andpostal abbreviation[12]                                    \
  Name andpostal abbreviation[12] Name andpostal abbreviation[12].1   
0            District of Columbia                                DC   

        Established Population[14] Total area[15]      Land area[15]       \
        Established Population[14]            mi2  km2           mi2  km2   
0  Jul 16, 1790[17]         689545             68  176            61  158   

  Water area[15]     Numberof Reps.  
             mi2 km2 Numberof Reps.  
0              7  18           1[E]

In [26]:
# Information about District of Columbia

DC = DC.iloc[:, [0, 1, 3, 4, 6]]
DC.columns = ['Name', 'Abbreviation', 'Population', 'Total_area', 'Land_area']

In [27]:
DC.dtypes

Name            object
Abbreviation    object
Population       int64
Total_area       int64
Land_area        int64
dtype: object

In [28]:
states = pd.merge(states, DC, how='outer')

Adding information about USA territories

In [29]:
territories = tables_states_list[2]
territories.head(1)

Name andpostal abbreviation[12]                                    \
  Name andpostal abbreviation[12] Name andpostal abbreviation[12].1   
0                  American Samoa                                AS   

         Capital Acquired[19]          Territorial status[20] Population[F]  \
         Capital Acquired[19]          Territorial status[20] Population[F]   
0  Pago Pago[21]         1900  Unincorporated, unorganized[G]    57,400[22]   

  Total area[15]       Land area[15]      Water area[15]       Numberof Reps.  
             mi2   km2           mi2  km2            mi2   km2 Numberof Reps.  
0            581  1505            76  198            505  1307           1[E]

In [30]:
%%capture

territories = territories.iloc[:, [0, 1, 5, 7, 9]]
territories.columns = ['Name', 'Abbreviation', 'Population', 'Total_area', 'Land_area']

# Removing coomments and useless commas
territories.Population = territories.Population\
        .astype('str')\
        .str.replace('\[..\]', '', regex=True)\
        .str.replace(',', '', regex=False)\
        .astype('int64')

In [31]:
territories.dtypes

Name            object
Abbreviation    object
Population       int64
Total_area       int64
Land_area        int64
dtype: object

In [32]:
states = pd.merge(states, territories, how='outer')

Adding information about Canada states

In [33]:
# States of Canada

canada = pd.read_html('https://en.wikipedia.org/wiki/Provinces_and_territories_of_Canada')[1]
canada = canada.iloc[:-1, [0, 1, 6, 7, 8]]
canada.columns = ['Name', 'Abbreviation', 'Population', 'Total_area', 'Land_area']
canada.Name = canada.Name.str.replace('[b]', '', regex=False)

In [34]:
states = pd.merge(states, canada, how='outer')

Adding information about Canada territories

In [35]:
# Territories of Canada

canada_territories = pd.read_html('https://en.wikipedia.org/wiki/Provinces_and_territories_of_Canada')[2]
canada_territories = canada_territories.iloc[:-1, [0, 1, 6, 7, 8]]
canada_territories.columns = ['Name', 'Abbreviation', 'Population', 'Total_area', 'Land_area']

In [36]:
states = pd.merge(states, canada_territories, how='outer')

In [37]:
states

,Name,Abbreviation,Population,Total_area,Land_area
0,Alabama,AL,5024279,135767,131171
1,Alaska,AK,733391,1723337,1477953
2,Arizona,AZ,7151502,295234,294207
3,Arkansas,AR,3011524,137732,134771
4,California,CA,39538223,423967,403466
...,...,...,...,...,...
64,Alberta,AB,4444277,642317,19531
65,Newfoundland and Labrador,NL,520286,373872,31340
66,Northwest Territories,NT,44991,1183085,163021
67,Yukon,YT,42596,474391,8052


In [38]:
states.to_csv('states.csv')

# <span style="color:blue">Data reading from local files</span>

In [39]:
ufo_by_date = pd.read_csv('ufo_raw_date.csv', index_col=0)

In [40]:
ufo_by_shape = pd.read_csv('ufo_raw_shape.csv', index_col=0)

In [41]:
states = pd.read_csv('states.csv', index_col=0)

# <span style="color:blue">Preliminary cleaning of the data</span>

Number of rows of the dataframes obtained via import by shape and by date is the same. Additional column in `ufo_by_date` is the manually introduced `Date_label` column.

In [ ]:
ufo_by_date.shape, ufo_by_shape.shape

In [ ]:
pd.merge(left=ufo_by_date, right=ufo_by_shape, how='outer', indicator=True)\
    .query('_merge != "both"').shape

In [ ]:
pd.merge(left=ufo_by_date, right=ufo_by_shape, how='outer', indicator=True)\
    .query('_merge == "both"').shape

Datasets collected by shape and by date are identical (result of the first merge is empty), but there are a few duplicates (esult of the second merge is longer than the initial dataframe).

We will further work with `ufo_raw_date` to keep "manual" date labels.

In [ ]:
# making a copy to preserve source data
ufo = ufo_by_date.copy()

In [ ]:
ufo

In [ ]:
ufo.dtypes

We should consider converting `Date / Time` and `Posted` columns to datetime format.
On top of that, the column name with slash and spaces is not convenient.

In [ ]:
ufo.rename(columns={'Date / Time': 'Observed'}, inplace=True)

In [ ]:
round(sum(ufo.Date_label == 'UNSPECIFIED / APPROXIMATE') / ufo.shape[0] , 4)

Parsing of poorly formatted dates/times is a complicated issue, but these are less than 0.3% of the observations. Hence, the solution is to remove such rows.

Then, we will remove too old observations (say, before 1900). This will resolve the issue of limited range of the `pd.datetime` type; moreover, these observations seem not reliable and too scarce to affect the conclusions.

Since year is coded by two last digits, we will extract it from the earlier introduced `Date_label` column. A new `Year` column will be created instead of just query because we will then use it to modify the `Observed` values

In [ ]:
ufo = ufo.query('Date_label != "UNSPECIFIED / APPROXIMATE"')

# this is divided into two linea to suppress warning
years = ufo.Date_label.apply(lambda x: int(x[-4:]))
ufo['Year'] = years

ufo = ufo.query("Year >= 1900")

Without the following insertion of complete year, '68' would be converted to '2068'

In [ ]:
times = []

for row in ufo.itertuples():
    date_complete_year = re.sub('/(.. )', '/' + str(row.Year) + ' ', row.Observed)
    times.append(date_complete_year)

ufo['Observed'] = pd.to_datetime(times, format="%m/%d/%Y %H:%M", errors='coerce')

Still, about 1.3% of the dates/times were not complete, even though not marked as 'UNSPECIFIED / APPROXIMATE"'  

An example of such entry is given below (day and time are missing)

In [ ]:
round(ufo.Observed.isnull().sum() / ufo.shape[0], 3)

In [ ]:
pd.DataFrame({
                'working': ufo.loc[134435, :],
                'as imported': ufo_by_date.loc[134435, :]
             })

In the case of `Posted` date, conversion to `datetime` will be ambiguous anyway, but we will do it (maybe we will not use this column at all)

In [ ]:
ufo.Posted = pd.to_datetime(ufo.Posted, format="%m/%d/%y", errors='coerce')

In [ ]:
ufo.dtypes

We don't need Date_label and Year columns any more

In [ ]:
ufo.drop(columns=['Date_label', 'Year'], inplace=True)

Checking non-null data

In [ ]:
ufo.info()

number of totally incomplete observations (5 is for Observed, City, State, Shape, and Duration columns

In [ ]:
ufo[ufo[['Observed', 'City', 'Shape', 'State', 'Duration']].isnull().sum(axis=1) == 5].shape[0]

filtering them out

In [ ]:
ufo = ufo[ufo[['Observed', 'City', 'Shape', 'State', 'Duration']].isnull().sum(axis=1) < 5]

In [ ]:
ufo.shape

About 99.7% of the data were retained at this step

In [ ]:
round(ufo.shape[0] / ufo_by_date.shape[0], 4)

Working with States column - we don't need states outside USA

In [ ]:
states.tail(12)

In [ ]:
ufo.query('State not in @states.Abbreviation.values').query('not State.isnull()').State.value_counts()

* PQ is another assignment of Quebec (QC)
* NF stands for Newfoundland and Labrador (which is NL in the dataset)
* SA is for Sounth Australia
* YK seems to stand for Yukon (YT)
* Fl is misprinted FL (Florida)
* QB seems to be Quebec (QC) as well
* M0 is misprinted MO
* Ca is misprinted CA

In [ ]:
new_state = {
                'State': {'PQ': 'QC',
                          'NF': 'NL',
                          'SA': np.nan,
                          'YK': 'YT',
                          'Fl': 'FL',
                          'QB': 'QC',
                          'M0': 'MO',
                          'Ca': 'CA'}
            }

ufo = ufo.replace(new_state)

In [ ]:
ufo.query('State not in @states.Abbreviation.values').query('not State.isnull()')

Now it is time to turn to duplicates

Four cases have complete duplicates

In [ ]:
ufo[ufo.duplicated(['Observed', 'City', 'State', 'Shape', 'Duration', 'Summary', 'Posted'], keep=False)]\
    .sort_values(by=['Observed', 'City', 'State', 'Shape', 'Duration', 'Summary', 'Posted'])

Four more observations differ in the Posted date

In [ ]:
ufo[ufo.duplicated(['Observed', 'City', 'State', 'Shape', 'Duration', 'Summary'], keep=False)]\
    .sort_values(by=['Observed', 'City', 'State', 'Shape', 'Duration', 'Summary', 'Posted'])

Quite many observations share place, time, and duration, but Summary is different.

Arbitrary inspection of several first cases show that there are minor textual differences in the Summary field. It seems dafe to consider them true duplicates.

No more investigation on duplicates on place, time and duration will be performed - let us consider that if at least duration is different, this can indicate multiple objects or multiple (=more reliable) observations of the same object.

In [ ]:
ufo[ufo.duplicated(['Observed', 'City', 'State', 'Shape', 'Duration'], keep=False)]\
    .sort_values(by=['Observed', 'City', 'State', 'Shape', 'Duration', 'Summary', 'Posted'], ascending=False)

Hence, the decision is to remove duplicates basing on 'Observed', 'City', 'State', 'Shape', 'Duration' fields:

In [ ]:
# this is not needed since we will drop duplicates later

#ufo.drop_duplicates(['Observed', 'City', 'State', 'Shape', 'Duration'], keep='first', inplace=True)

Upon this step, we have kept >99% of the observations.

Many of them (about 13.6%) have NaN in at least one of the columns, but let us consider these as partial potentially useful information.

In [ ]:
round(ufo.shape[0] / ufo_by_date.shape[0], 4)

In [ ]:
round((ufo.shape[0] - ufo.dropna().shape[0]) / ufo.shape[0], 3)

Plan:
* **DONE** convert `Date / Time` to datetime
* **DONE** break datetime into date and time?
* **DONE** convert `posted` to date
* **DONE** remove duplicates
* aggregate common shapes
* remove duplicates again (probably not perform first deletion?)
* filter off old rare occasions (20%?)

In [ ]:
ufo.Shape.str.lower().value_counts().sort_index()

It seems reasonable to 
1) ignore case and 
2) consider (other synonims are possible but questionable):
* 'changed' = 'changing'
* 'cigar' = 'cylinder'
* 'delta' = 'triangular' = 'triangle'
* 'circle' = 'disk' = 'round' = 'sphere'
* 'flare' = 'light'
* 'other' = 'unknown'
* 'egg' = 'oval'
and
3) set NaN to 'unknown'

In [ ]:
ufo.Shape = ufo.Shape.str.lower()

new_shape = {
                'Shape': {'changed': 'changing',
                          'cigar': 'cylinder',
                          'delta': 'triangle',
                          'triangular': 'triangle',
                          'disk': 'circle',
                          'round': 'circle',
                          'sphere': 'circle',
                          'flash': 'light',
                          'other': 'unknown',
                          'egg': 'oval',
                          np.nan: 'unknown'}
            }

ufo = ufo.replace(new_shape)

In [ ]:
ufo.Shape.value_counts()

In [ ]:
ufo[ufo.duplicated(['Observed', 'City', 'State', 'Shape', 'Duration'], keep=False)]\
    .sort_values(by=['Observed', 'City', 'State', 'Shape', 'Duration', 'Summary', 'Posted'], ascending=False)

Since several additional duplicates have appeared at this step, we will again drop them

In [ ]:
# not actually again - judt once here

ufo.drop_duplicates(['Observed', 'City', 'State', 'Shape', 'Duration'], keep='first', inplace=True)

In [ ]:
# saving cleaned dataframe in case we distort it occasionally during analysis
ufo.to_csv("ufo_for_analysis.csv")

# <span style="color:blue">Exploratory data analysis</span>

Plan:
* **DONE** Fractions of observations by shape
* **DONE** Line Plot of observations by year
* **DONE** Line Plot of observations by year and shape
* **DONE** Barplot of observations by state (raw and normalized by state population)
* Top counts by state + city

In [ ]:
# start analysis from scratch
ufo = pd.read_csv('ufo_for_analysis.csv', index_col=0, parse_dates=['Observed', 'Posted'])

In [ ]:
ufo_per_shape = ufo.groupby('Shape')\
        .agg(percentage =('Summary', lambda x: x.size / ufo.shape[0] * 100))\
        .sort_values(by='percentage', ascending=False)\
        .round(2)

idx = ufo_per_shape.index.tolist()
idx.remove('unknown')
ufo_per_shape = ufo_per_shape.reindex(idx + ['unknown'])

#Seaborn barplot
sns.set_style("whitegrid")

sns.set(font_scale=1.1)

bar,ax = plt.subplots(figsize=(12,8))
ax = sns.barplot(x=ufo_per_shape.index,
                 y='percentage',
                 data=ufo_per_shape,
                 ci=None,
                 palette="muted",
                 orient='v', )
ax.set_title("Distribution of UFO observed by shape", fontsize=15)
ax.set_xlabel ("UFO shape")
ax.set_ylabel ("Percentage")
ax.tick_params(axis='x', rotation=45)
# calculate the percentages and annotate the sns barplot
for rect in ax.patches:
    ax.text(rect.get_x(),rect.get_height() + 0.1,"%.1f%%"% rect.get_height(), weight='bold' );

In [ ]:
ufo_per_year = ufo.copy()
ufo_per_year.index = ufo.Observed
ufo_per_year = ufo_per_year.resample(rule='Y').Summary.count()

sns.set_style("whitegrid")
sns.set(font_scale=1.1)

plt.figure(figsize=(12,8))
ax = sns.lineplot(x=ufo_per_year.index,
                 y=ufo_per_year,
                 palette="muted")

ax.set_title("Total UFO observed by year", fontsize=15)
ax.set_xlabel ("Year")
ax.set_ylabel ("Observations");

In [ ]:
ufo_per_year_shape = ufo.assign(Year = ufo.Observed.dt.year)

ufo_per_year_shape = ufo_per_year_shape.groupby(['Year', 'Shape'], as_index=False)\
        .agg({'Summary': 'count'})\
        .pivot('Year', columns='Shape', values='Summary')

ufo_per_year_shape['overall'] = ufo_per_year_shape.sum(axis=1)

ufo_per_year_shape = ufo_per_year_shape.reset_index()\
    .melt(id_vars = 'Year', var_name='Shape',  value_name='Observations')



sns.set_style("whitegrid")
sns.set(font_scale=1.1)

plt.figure(figsize=(12,8))
ax = sns.lineplot(data=ufo_per_year_shape,
                  x='Year',
                  y='Observations',
                  hue='Shape',
                 palette="muted")

ax.set_title("UFO observed by year", fontsize=15)
ax.set_xlabel ("Year")
ax.set_ylabel ("Observations");

In [ ]:
ufo_per_year_shape = ufo.assign(Year = ufo.Observed.dt.year)

ufo_per_year_shape = ufo_per_year_shape.groupby(['Year', 'Shape'], as_index=False)\
        .agg({'Summary': 'count'})\
        .pivot('Year', columns='Shape', values='Summary')

ufo_per_year_shape['overall'] = ufo_per_year_shape.sum(axis=1)

ufo_per_year_shape = ufo_per_year_shape.reset_index()\
    .melt(id_vars = 'Year', var_name='Shape',  value_name='Observations')

ufo_per_year_shape = ufo_per_year_shape.query("Shape in ['overall', 'circle', 'light', 'triangle', 'fireball', 'oval', 'cylinder', 'formation']")\
            .query("1960 <= Year <= 2021")


sns.set_style("whitegrid")
sns.set(font_scale=1.1)

plt.figure(figsize=(12,8))
ax = sns.lineplot(data=ufo_per_year_shape,
                  x='Year',
                  y='Observations',
                  hue='Shape',
                 palette="muted")

ax.set_title("UFO observed by year", fontsize=15)
ax.set_xlabel ("Year")
ax.set_ylabel ("Observations");

In [ ]:
ufo_per_state = ufo.groupby('State', as_index=False)\
        .agg({'Summary': 'count'})\
        .rename(columns={'Summary': 'Observations'})\
        .sort_values(by='State', ascending=True)

#Seaborn barplot
sns.set_style("whitegrid")

sns.set(font_scale=1)

bar,ax = plt.subplots(figsize=(14,10))
ax = sns.barplot(x='State',
                 y='Observations',
                 data=ufo_per_state,
                 ci=None,
                 palette="muted",
                 orient='v')
ax.set_title("Distribution of UFO observed by state", fontsize=15)
ax.set_xlabel ("State")
ax.set_ylabel ("Number of observations")
ax.tick_params(axis='x', rotation=90)

Distribution by state is strongly uneven; let us normalize it to the state population and area

In [ ]:
ufo_per_state = pd.merge(left=ufo_per_state, 
                         right=states, how='inner', 
                         left_on='State', 
                         right_on='Abbreviation')

In [ ]:
ufo_per_state['Observations_per_population'] = ufo_per_state.Observations / ufo_per_state.Population
ufo_per_state['Observations_per_area'] = ufo_per_state.Observations / ufo_per_state.Total_area
ufo_per_state['Observations_per_land_area'] = ufo_per_state.Observations / ufo_per_state.Land_area

In [ ]:
sns.set_style("whitegrid")

sns.set(font_scale=1)

bar,ax = plt.subplots(figsize=(14,10))
ax = sns.barplot(x='State',
                 y='Observations_per_population',
                 data=ufo_per_state,
                 ci=None,
                 palette="muted",
                 orient='v')
ax.set_title("Distribution of UFO observed by state normalized by current population", fontsize=15)
ax.set_xlabel ("State")
ax.set_ylabel ("Number of observations per person")
ax.tick_params(axis='x', rotation=90)

In [ ]:
sns.set_style("whitegrid")

sns.set(font_scale=1)

bar,ax = plt.subplots(figsize=(14,10))
ax = sns.barplot(x='State',
                 y='Observations_per_area',
                 data=ufo_per_state,
                 ci=None,
                 palette="muted",
                 orient='v')
ax.set_title("Distribution of UFO observed by state normalized by total area", fontsize=15)
ax.set_xlabel ("State")
ax.set_ylabel ("Number of observations per sq.km")
ax.tick_params(axis='x', rotation=90)

In [ ]:
sns.set_style("whitegrid")

sns.set(font_scale=1)

bar,ax = plt.subplots(figsize=(14,10))
ax = sns.barplot(x='State',
                 y='Observations_per_land_area',
                 data=ufo_per_state,
                 ci=None,
                 palette="muted",
                 orient='v')
ax.set_title("Distribution of UFO observed by state normalized by land area", fontsize=15)
ax.set_xlabel ("State")
ax.set_ylabel ("Number of observations per sq.km")
ax.tick_params(axis='x', rotation=90)

In [ ]:
def z_score(column):
    column.replace([np.inf, -np.inf], np.nan, inplace=True)
    return (column - column.mean())/column.std()


ufo_per_state['Observations_per_population_std'] = z_score(ufo_per_state['Observations_per_population'])
ufo_per_state['Observations_per_area_std'] = z_score(ufo_per_state['Observations_per_area'])
ufo_per_state['Observations_per_land_area_std'] = z_score(ufo_per_state['Observations_per_land_area'])

std_observations = ufo_per_state.loc[:, ['State', 'Observations_per_population_std', 'Observations_per_area_std', 'Observations_per_land_area_std']]

std_observations = pd.melt(frame=std_observations, 
        id_vars='State',
        value_vars=['Observations_per_population_std', 'Observations_per_area_std', 'Observations_per_land_area_std'],
        )

In [ ]:
plt.figure(figsize=(8,6))
sns.boxplot(x="value", y="variable", data=std_observations, color="c")
sns.despine(trim=True)

In [ ]:
ufo_per_state

In [ ]:
ufo.dtypes